In [1]:
!git clone https://github.com/CASIA-IVA-Lab/FastSAM.git
%pip  install -r FastSAM/requirements.txt
# install CLIP
%pip  install git+https://github.com/openai/CLIP.git
# install SAM
%pip  install git+https://github.com/facebookresearch/segment-anything.git
# install other dependencies
%pip  install roboflow supervision jupyter_bbox_widget
%pip install roboflow

Cloning into 'FastSAM'...
remote: Enumerating objects: 1221, done.
remote: Counting objects: 100% (289/289), done.
remote: Compressing objects: 100% (93/93), done.
remote: Total 1221 (delta 229), reused 205 (delta 196), pack-reused 932
Receiving objects: 100% (1221/1221), 72.46 MiB | 40.82 MiB/s, done.
Resolving deltas: 100% (486/486), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.7/611.7 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 12.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 222.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from roboflow import Roboflow
rf = Roboflow(api_key="OZmt7veKofDCRjgyQ5XE")
project = rf.workspace("csilab").project("csilab-braintumor")
dataset = project.version(3).download("yolov8")

import os
HOME = os.getcwd()
print("HOME:", HOME)

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.0.120, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to CsiLab-BrainTumor-3 in yolov8:: 100%|██████████| 618/618 [00:00<00:00, 10640.98it/s]

HOME: /content


In [2]:
from roboflow import Roboflow
rf = Roboflow(api_key="OZmt7veKofDCRjgyQ5XE")
project = rf.workspace("csilab").project("csilab-braintumor-detection")
dataset = project.version(3).download("yolov8")


import os
HOME = os.getcwd()
print("HOME:", HOME)


loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.0.120, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to CsiLab-BrainTumor-Detection-3 in yolov8:: 100%|██████████| 614/614 [00:00<00:00, 7137.93it/s]

HOME: /content


In [3]:
!mkdir -p {HOME}/weights
!wget -P {HOME}/weights -q https://huggingface.co/spaces/An-619/FastSAM/resolve/main/weights/FastSAM.pt


In [4]:
FAST_SAM_CHECKPOINT_PATH = "/content/weights/FastSAM.pt"

In [5]:
%cd {HOME}/FastSAM

import os
import cv2
import torch
import roboflow
import base64

import supervision as sv
import numpy as np

from roboflow import Roboflow
from fastsam import FastSAM, FastSAMPrompt
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

/content/FastSAM


In [6]:
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

fast_sam = FastSAM(FAST_SAM_CHECKPOINT_PATH)

In [7]:
def convert_masks_to_yolo_labels(masks):

  labels = []
  ob_id = 0
  for mask, in zip(masks):
    # Move the mask tensor to the CPU
    mask_cpu = mask.cpu()

    # Convert the mask tensor to a NumPy array
    mask_np = mask_cpu.numpy()

    bounding_box = cv2.boundingRect(mask_np.astype(np.uint8))

    x_center = bounding_box[0] + bounding_box[2] // 2
    y_center = bounding_box[1] + bounding_box[3] // 2
    width = bounding_box[2]
    height = bounding_box[3]

    x_center = x_center / mask.shape[1]
    y_center = y_center / mask.shape[0]
    width = width / mask.shape[1]
    height = height / mask.shape[0]

    label = [ob_id, x_center, y_center, width, height]
    labels.append(label)
    ob_id+=1


  return labels

In [8]:
#inferencia em todas as imagens de treino do dataset
import os
path = dataset.location + "/train/images/"
Output_Path = dataset.location +"/output/"

if not os.path.exists(Output_Path):
    os.makedirs(Output_Path)
    print("Criado diretorio: ", Output_Path)

for IMAGE in os.listdir(path):
   if (IMAGE.endswith(".jpg")):
      IMAGE_PATH = path+"/"+IMAGE
      results = fast_sam(
        source=IMAGE_PATH,
        device=DEVICE,
        retina_masks=True,
        imgsz=1024,
        conf=0.5,
        iou=0.5)
      prompt_process = FastSAMPrompt(IMAGE_PATH, results, device=DEVICE)
      masks = prompt_process.everything_prompt()
      prompt_process.plot(annotations=masks, output_path = Output_Path + IMAGE)
      #torch.save(masks, (Output_Path + IMAGE).split(".")[0] + ".pt")
      yolo_labels = convert_masks_to_yolo_labels(masks)

      '''out_file_path = (Output_Path + IMAGE).split(".jpg")[0] + ".txt"
      with open(out_file_path, "w") as out_file:
          for label in yolo_labels:
              out_file.write(" ".join(map(str, label)) + "\n")'''
      #separando por objetos detectatos
      for label in yolo_labels:
        out_file_path = (Output_Path + IMAGE).split(".jpg")[0]+ "object_" + str(label[0]) + ".txt"
        out_file = open(out_file_path, "w")
        out_file.write(" ".join(map(str, label)) + "\n")



Criado diretorio:  /content/CsiLab-BrainTumor-Detection-3/output/



image 1/1 /content/CsiLab-BrainTumor-Detection-3/train/images/42_jpg.rf.5cb629c683cc6e93575fda8eefd6e750.jpg: 1024x1024 4 objects, 157.9ms
Speed: 8.0ms preprocess, 157.9ms inference, 46.1ms postprocess per image at shape (1, 3, 1024, 1024)

image 1/1 /content/CsiLab-BrainTumor-Detection-3/train/images/Te-me_0028_jpg.rf.f92f9e5c8684c0c95f0183a610b12822.jpg: 1024x832 5 objects, 141.0ms
Speed: 3.9ms preprocess, 141.0ms inference, 2.7ms postprocess per image at shape (1, 3, 1024, 1024)

image 1/1 /content/CsiLab-BrainTumor-Detection-3/train/images/Te-me_0054_jpg.rf.137883941c66700e9bfb3f398284a1be.jpg: 1024x960 1 object, 158.0ms
Speed: 4.9ms preprocess, 158.0ms inference, 3.7ms postprocess per image at shape (1, 3, 1024, 1024)

image 1/1 /content/CsiLab-BrainTumor-Detection-3/train/images/Te-me_0107_jpg.rf.1a245930d18e91aa003d966f0bd83d55.jpg: 1024x960 2 objects, 151.0ms
Speed: 5.7ms preprocess, 151.0ms inference, 2.7ms postprocess per image at shape (1, 3, 1024, 1024)

image 1/1 /content

No object detected.


image 1/1 /content/CsiLab-BrainTumor-Detection-3/train/images/65_jpg.rf.c34f16c557e5c23e717b22282584f7a2.jpg: 1024x1024 2 objects, 170.0ms
Speed: 5.7ms preprocess, 170.0ms inference, 6.4ms postprocess per image at shape (1, 3, 1024, 1024)

image 1/1 /content/CsiLab-BrainTumor-Detection-3/train/images/Te-me_0065_jpg.rf.12e2dd8dd599649cb66ba9ccba4d16c9.jpg: 1024x864 5 objects, 151.0ms
Speed: 3.9ms preprocess, 151.0ms inference, 3.0ms postprocess per image at shape (1, 3, 1024, 1024)

image 1/1 /content/CsiLab-BrainTumor-Detection-3/train/images/Te-me_0134_jpg.rf.d625d4ac064c1072700a526d097eb104.jpg: 1024x1024 2 objects, 164.8ms
Speed: 4.9ms preprocess, 164.8ms inference, 3.2ms postprocess per image at shape (1, 3, 1024, 1024)

image 1/1 /content/CsiLab-BrainTumor-Detection-3/train/images/51_jpg.rf.de1b3448da41d4244e7598fdaaf4e38a.jpg: 1024x1024 2 objects, 167.3ms
Speed: 4.9ms preprocess, 167.3ms inference, 3.4ms postprocess per image at shape (1, 3, 1024, 1024)

image 1/1 /content/CsiLab

In [7]:
def convert_masks_to_yolo_labels(masks):

  labels = []
  ob_id = 0
  for mask, in zip(masks):
    # Move the mask tensor to the CPU
    #mask_cpu = mask.cpu()

    # Convert the mask tensor to a NumPy array
    #mask_np = mask_cpu.numpy()

    bounding_box = cv2.boundingRect(mask.astype(np.uint8))

    x_center = bounding_box[0] + bounding_box[2] // 2
    y_center = bounding_box[1] + bounding_box[3] // 2
    width = bounding_box[2]
    height = bounding_box[3]

    x_center = x_center / mask.shape[1]
    y_center = y_center / mask.shape[0]
    width = width / mask.shape[1]
    height = height / mask.shape[0]

    label = [ob_id, x_center, y_center, width, height]
    labels.append(label)
    labels.append(';')
    ob_id+=1


  return labels

In [9]:
def calculate_iou(box1, box2):
      # box1 e box2 devem ser no formato [x, y, width, height]
    try:
      # Extrair as coordenadas das bounding boxes
      x1, y1, w1, h1 = box1
      x2, y2, w2, h2 = box2

      # Calcular as coordenadas dos cantos superiores esquerdos e inferiores direitos
      x1_min, y1_min, x1_max, y1_max = x1, y1, x1 + w1, y1 + h1
      x2_min, y2_min, x2_max, y2_max = x2, y2, x2 + w2, y2 + h2

      # Calcular a área da interseção
      x_intersection = max(0, min(x1_max, x2_max) - max(x1_min, x2_min))
      y_intersection = max(0, min(y1_max, y2_max) - max(y1_min, y2_min))
      intersection_area = x_intersection * y_intersection

      # Calcular a área da união
      area1 = w1 * h1
      area2 = w2 * h2
      union_area = area1 + area2 - intersection_area

      # Calcular o IoU
      iou = intersection_area / union_area

      return iou
    except Exception as e:
      return 0

In [16]:
import os
from statistics import mean
result_iou = []
source = '/content/CsiLab-BrainTumor-Detection-3/output'
label = '/content/CsiLab-BrainTumor-Detection-3/train/labels'
fileaux = ''
for filename in os.listdir(f"{source}"):
  box2 = []
  box1 = []
  if (filename.endswith(".txt")):

    file_1 = open(f"{source}/{filename}")
    for values in file_1.read().split():
      box1.append(float(values))
    #print(box1[1:5])


    filename_2 = filename.split("object_")[0] + ".txt"
    file_2 = open(f"{label}/{filename_2}")

    for values in file_2.read().split():
      box2.append(float(values))
    #print(box2[1:5])
    #print(filename)

    #result_iou.append(float(calculate_iou(box1[1:5], box2[1:5])))
    iou = calculate_iou(box1[1:5], box2[1:5])
    #print(filename, iou)

    if fileaux == filename_2:
        result_iou[-1] = max(result_iou[-1], iou)
    else:
        if iou >= 0.1: #desconsiderar inferencias muito erradas.
          result_iou.append(iou)


    fileaux = filename_2
print("Media de Iou do Fastsam", mean(result_iou),f"ou {mean(result_iou)*100}%")

Media de Iou do Fastsam 0.6996150975040285 ou 69.96150975040285%
